In [53]:
def combineSimilarTables(data):
    dfs = []
    dfs_by_column_count = {}
    
    for i, table in enumerate(data):
        df = table.df
        num_columns = len(df.columns)
    
        # Set the first row as the header (column names) and drop it from the data
        df.columns = df.iloc[0]  # Set the first row as column names
        df = df[1:]  # Drop the first row from the data
    
        # Get number of columns
        num_columns = len(df.columns)
        
        if num_columns in dfs_by_column_count:
            # Combine the existing DataFrame with the new one
            dfs_by_column_count[num_columns] = pd.concat([dfs_by_column_count[num_columns], df], ignore_index=True)
        else:
            # Initialize the entry with the current DataFrame
            dfs_by_column_count[num_columns] = df
    return dfs_by_column_count

In [9]:
import camelot
import pandas as pd
import numpy as np

def extract_tables_from_pdf(pdf_path, pages='all'):
    """
    Extract tables from PDF file using Camelot.
    
    Args:
        pdf_path (str): Path to PDF file
        pages (str): Page numbers to process (e.g., '1,2,3' or 'all')
    
    Returns:
        list: List of pandas DataFrames, one for each table found
    """
    try:
        # Extract tables using both lattice and stream modes
        tables_lattice = camelot.read_pdf(pdf_path, pages=pages, flavor='lattice')
        tables_stream = camelot.read_pdf(pdf_path, pages=pages, flavor='stream')
        
        print(f"Found {len(tables_lattice)} tables using lattice mode")
        print(f"Found {len(tables_stream)} tables using stream mode")
        
        # # Convert to list of dataframes
        # dfs_lattice = [table.df for table in tables_lattice]
        # dfs_stream = [table.df for table in tables_stream]
        
        # # Combine results from both modes
        # # all_dfs = dfs_lattice + dfs_stream
        
        return [tables_lattice, tables_stream]

    except Exception as e:
        print(f"Error extracting tables: {str(e)}")
        return []

if __name__ == "__main__":
    # Example with multiple tables in a PDF
    pdf_path = "classification_model/loss_runs/input/Loss_Run___len stoler 8-24.pdf"

    # Extract tables
    raw_tables = extract_tables_from_pdf(pdf_path)

Error extracting tables: Ghostscript is not installed. You can install it using the instructions here: https://camelot-py.readthedocs.io/en/master/user/install-deps.html


In [56]:
tables_lattice, tables_stream = raw_tables
stream_dfs = combineSimilarTables(tables_stream)

AttributeError: 'DataFrame' object has no attribute 'df'

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import camelot  # For table extraction from PDFs
import pdfplumber  # For general PDF text extraction
from sentence_transformers import SentenceTransformer  # For semantic similarity
from sklearn.metrics.pairwise import cosine_similarity
import re
from datetime import datetime
import logging
from typing import List, Dict, Tuple, Optional
import spacy

class PDFTableExtractor:
    def __init__(self):
        # Initialize models and target columns
        self.target_columns = [
            'location_id', 'carrier', 'claim_number', 'status', 'loss_cause',
            'occurred_at', 'expense', 'incurred', 'recovery', 'loss_amount', 'description'
        ]
        
        # Load models
        self.nlp = spacy.load("en_core_web_lg")  # For general NLP tasks
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # For semantic similarity
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # Calculate embeddings for target columns once
        self.target_embeddings = self.sentence_model.encode(self.target_columns)

    def extract_tables_from_pdf(self, pdf_path: str) -> List[pd.DataFrame]:
        """
        Extract tables from PDF using multiple methods and combine results.
        """
        tables = []
        
        # Try Camelot first (good for structured tables)
        try:
            camelot_tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
            tables.extend([table.df for table in camelot_tables])
        except Exception as e:
            self.logger.warning(f"Camelot extraction failed: {str(e)}")

        # Fallback to pdfplumber (better for less structured tables)
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    extracted_tables = page.extract_tables()
                    for table in extracted_tables:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        tables.append(df)
        except Exception as e:
            self.logger.warning(f"PDFPlumber extraction failed: {str(e)}")

        return tables

    def preprocess_column_names(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Clean and standardize column names.
        """
        def clean_column(col):
            # Convert to lowercase and remove special characters
            col = str(col).lower().strip()
            col = re.sub(r'[^a-z0-9\s]', '', col)
            col = re.sub(r'\s+', '_', col)
            return col

        df.columns = [clean_column(col) for col in df.columns]
        return df

    def find_best_column_matches(self, df: pd.DataFrame) -> Dict[str, str]:
        """
        Use semantic similarity to match source columns to target columns.
        """
        column_mappings = {}
        source_embeddings = self.sentence_model.encode(df.columns.tolist())
        
        # Calculate similarity matrix
        similarity_matrix = cosine_similarity(source_embeddings, self.target_embeddings)
        
        # Find best matches
        for target_idx, target_col in enumerate(self.target_columns):
            best_match_idx = np.argmax(similarity_matrix[:, target_idx])
            source_col = df.columns[best_match_idx]
            score = similarity_matrix[best_match_idx, target_idx]
            
            # Only map if similarity is above threshold
            if score > 0.5:
                column_mappings[target_col] = source_col
                
        return column_mappings

    def extract_value(self, text: str, value_type: str) -> Optional[str]:
        """
        Extract and normalize specific types of values.
        """
        if pd.isna(text):
            return None
            
        text = str(text).strip()
        
        if value_type == 'amount':
            # Extract monetary amounts
            amount_match = re.search(r'[\$]?([0-9,]+\.?[0-9]*)', text)
            if amount_match:
                return float(amount_match.group(1).replace(',', ''))
                
        elif value_type == 'date':
            # Try multiple date formats
            date_patterns = [
                '%Y-%m-%d', '%m/%d/%Y', '%d/%m/%Y', 
                '%b %d %Y', '%B %d %Y', '%Y/%m/%d'
            ]
            for pattern in date_patterns:
                try:
                    return datetime.strptime(text, pattern).strftime('%Y-%m-%d')
                except ValueError:
                    continue
                    
        return text

    def process_table(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Process and normalize extracted table data.
        """
        # Preprocess column names
        df = self.preprocess_column_names(df)
        
        # Find best column matches
        column_mappings = self.find_best_column_matches(df)
        
        # Create new DataFrame with target columns
        result_df = pd.DataFrame(columns=self.target_columns)
        
        # Map and process values
        for target_col in self.target_columns:
            if target_col in column_mappings:
                source_col = column_mappings[target_col]
                
                # Apply appropriate extraction based on column type
                if target_col in ['expense', 'incurred', 'recovery', 'loss_amount']:
                    result_df[target_col] = df[source_col].apply(
                        lambda x: self.extract_value(x, 'amount')
                    )
                elif target_col == 'occurred_at':
                    result_df[target_col] = df[source_col].apply(
                        lambda x: self.extract_value(x, 'date')
                    )
                else:
                    result_df[target_col] = df[source_col].apply(str)
            
        return result_df

    def process_pdf(self, pdf_path: str) -> pd.DataFrame:
        """
        Main method to process a PDF and return structured data.
        """
        # Extract tables
        tables = self.extract_tables_from_pdf(pdf_path)
        
        if not tables:
            self.logger.warning(f"No tables found in {pdf_path}")
            return pd.DataFrame(columns=self.target_columns)
        
        # Process each table and combine results
        processed_tables = []
        for table in tables:
            processed_df = self.process_table(table)
            if not processed_df.empty:
                processed_tables.append(processed_df)
        
        # Combine all processed tables
        if processed_tables:
            final_df = pd.concat(processed_tables, ignore_index=True)
            return final_df
        
        return pd.DataFrame(columns=self.target_columns)

def process_multiple_pdfs(pdf_directory: str, output_path: str):
    """
    Process multiple PDFs and combine results into a single CSV file.
    """
    extractor = PDFTableExtractor()
    pdf_files = Path(pdf_directory).glob('*.pdf')
    all_results = []
    
    for pdf_file in pdf_files:
        try:
            result_df = extractor.process_pdf(str(pdf_file))
            if not result_df.empty:
                # Add source file information
                result_df['source_file'] = pdf_file.name
                all_results.append(result_df)
        except Exception as e:
            logging.error(f"Error processing {pdf_file}: {str(e)}")
    
    if all_results:
        # Combine all results
        final_df = pd.concat(all_results, ignore_index=True)
        # Save to CSV
        final_df.to_csv(output_path, index=False)
        logging.info(f"Results saved to {output_path}")
    else:
        logging.warning("No data extracted from PDFs")

# Example usage
if __name__ == "__main__":
    # Process a directory of PDFs
    process_multiple_pdfs(
        pdf_directory="./samples/input/",
        output_path="extracted_data.csv"
    )

ModuleNotFoundError: No module named 'sentence_transformers'

# PDF Plumber Test
- First, use extract tables to get the tables off PDF
- Then, convert to dataframes and attempt to clean
- Compare with that produced by camelot
- Concept: Fallback to LLM only if local installations fail to deliver expected results

In [14]:
import pdfplumber

# Path to the uploaded PDF
pdf_path = "./samples/input/Loss_Run___len stoler 8-24.pdf"

# Extract tables from the PDF
extracted_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        # Extract tables from the current page
        tables = page.extract_tables()
        for table in tables:
            extracted_data.append(table)

In [34]:
dfs = []
dfs_by_column_count = {}

for i, table in enumerate(extracted_data):
    df = pd.DataFrame(table)
    num_columns = len(df.columns)

    # Set the first row as the header (column names) and drop it from the data
    df.columns = df.iloc[0]  # Set the first row as column names
    df = df[1:]  # Drop the first row from the data

    # Get number of columns
    num_columns = len(df.columns)
    
    if num_columns in dfs_by_column_count:
        # Combine the existing DataFrame with the new one
        dfs_by_column_count[num_columns] = pd.concat([dfs_by_column_count[num_columns], df], ignore_index=True)
    else:
        # Initialize the entry with the current DataFrame
        dfs_by_column_count[num_columns] = df

In [33]:
display(dfs_by_column_count[13])

,Coverage,Sub Coverage,Val ID\nDescription,Claim\nNumber,Date of\nLoss,Status,Claimant\nName,Accident Narrative,Paid\nIndemnity,Paid\nExpense,Reserves\nTotal,Claim\nRecovery,Net\nIncurred
0,AUTOMOBILE,AUTO\nLIABILITY,NO FAULT BI\nMEDICAL,4340183468,20211116,Opened,Jackson-Smith\nIris,"""PIP LOSS"" IV driver\nexited the parked IV\nwh...",$0.00,"$2,926.89",$0.00,$0.00,"Total\n$2,926.89"
1,AUTOMOBILE,AUTO\nLIABILITY,NO FAULT BI\nMEDICAL,4340183468,20211116,Opened,Smith Edward,"""PIP LOSS"" IV driver\nexited the parked IV\nwh...","$46,264.65","$4,063.15","$7,990.94",$0.00,"$58,318.74"
2,AUTOMOBILE,AUTO\nPHYSICAL\nDAMAGE,COLLISION,1340152292,20211116,Closed,Westbury\nToyota,IV driver exited the\nparked IV when an OV\nth...,"$34,815.60",$165.30,$0.00,##########,"$6,879.49"
3,AUTOMOBILE,AUTO\nPHYSICAL\nDAMAGE,COLLISION,1510183901,20220326,Closed,Westbury\nToyota,V2 r/e V1,"$6,489.19",$131.85,$0.00,$0.00,"$6,621.04"
4,AUTOMOBILE,AUTO\nPHYSICAL\nDAMAGE,NEW VEHICLE\nCOMPREHENSIVE,1510178306,20211212,Closed,Westbury\nToyota,suspect came on lot\nand lit two vehicles on\n...,"$44,050.82",$132.50,$0.00,$0.00,"$44,183.32"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,AUTOMOBILE,AUTO\nPHYSICAL\nDAMAGE,SERVICE VEHICLE\nCOMP,1510207883,20231127,Closed,Len Stoler\nAutomotiv e,water leak in loaner\nvehicle,"$2,321.21",$125.00,$0.00,$0.00,"$2,446.21"
118,GENERAL\nLIABILITY,GARAGE\nLIABILITY,GARAGE LIABILITY -\nBI,4720162839,20240426,Opened,Apple Brett,Motorcyclist was riding\non motorcycle on East...,$0.00,$0.00,"$46,500.00",$0.00,"$46,500.00"
119,GENERAL\nLIABILITY,GARAGE\nLIABILITY,GARAGE LIABILITY -\nPD,4720158513,20240229,Closed,Unknown,Unknown.,$0.00,$0.00,$0.00,$0.00,$0.00
120,GENERAL\nLIABILITY,GARAGE\nLIABILITY,GARAGE LIABILITY -\nPD,4720162839,20240426,Opened,Apple Brett,Motorcyclist was riding\non motorcycle on East...,$0.00,$7.95,"$5,992.05",$0.00,"$6,000.00"
